# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 19 2023 9:04AM,264935,19,9566950-NJ,"Exact-Rx, Inc.",Released
1,Jun 19 2023 9:04AM,264935,19,9566957-NJ,"Exact-Rx, Inc.",Released
2,Jun 19 2023 9:03AM,264934,20,REL1-6464,Reliable 1 Laboratories LLC,Released
3,Jun 19 2023 9:03AM,264934,20,REL1-6466,Reliable 1 Laboratories LLC,Released
4,Jun 19 2023 9:03AM,264934,20,REL1-6467,Reliable 1 Laboratories LLC,Released
5,Jun 19 2023 9:03AM,264934,20,REL1-6468,Reliable 1 Laboratories LLC,Released
6,Jun 19 2023 9:03AM,264934,20,REL1-6469,Reliable 1 Laboratories LLC,Released
7,Jun 19 2023 9:03AM,264934,20,REL1-6471,Reliable 1 Laboratories LLC,Released
8,Jun 19 2023 9:03AM,264934,20,REL1-6480,Reliable 1 Laboratories LLC,Released
9,Jun 19 2023 9:03AM,264934,20,REL1-6481,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,264931,Released,18
9,264932,Released,46
10,264933,Released,1
11,264934,Released,15
12,264935,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264931,NaN,18.0
264932,NaN,46.0
264933,NaN,1.0
264934,NaN,15.0
264935,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264921,3.0,1.0
264925,1.0,0.0
264926,8.0,18.0
264927,1.0,0.0
264928,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264921,3,1
264925,1,0
264926,8,18
264927,1,0
264928,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264921,3,1
1,264925,1,0
2,264926,8,18
3,264927,1,0
4,264928,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264921,3,1
1,264925,1,
2,264926,8,18
3,264927,1,
4,264928,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc."
2,Jun 19 2023 9:03AM,264934,20,Reliable 1 Laboratories LLC
17,Jun 19 2023 8:49AM,264933,10,"Virtue Rx, LLC"
18,Jun 19 2023 8:48AM,264932,10,Hush Hush
64,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC"
82,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC"
83,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.
84,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.
85,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd"
111,Jun 19 2023 8:08AM,264925,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc.",,2
1,Jun 19 2023 9:03AM,264934,20,Reliable 1 Laboratories LLC,,15
2,Jun 19 2023 8:49AM,264933,10,"Virtue Rx, LLC",,1
3,Jun 19 2023 8:48AM,264932,10,Hush Hush,,46
4,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",,18
5,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC",,1
6,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.,,1
7,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.,1,
8,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd",8,18
9,Jun 19 2023 8:08AM,264925,19,"AdvaGen Pharma, Ltd",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc.",2,
1,Jun 19 2023 9:03AM,264934,20,Reliable 1 Laboratories LLC,15,
2,Jun 19 2023 8:49AM,264933,10,"Virtue Rx, LLC",1,
3,Jun 19 2023 8:48AM,264932,10,Hush Hush,46,
4,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18,
5,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC",1,
6,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.,1,
7,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.,,1
8,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd",18,8
9,Jun 19 2023 8:08AM,264925,19,"AdvaGen Pharma, Ltd",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc.",2,
1,Jun 19 2023 9:03AM,264934,20,Reliable 1 Laboratories LLC,15,
2,Jun 19 2023 8:49AM,264933,10,"Virtue Rx, LLC",1,
3,Jun 19 2023 8:48AM,264932,10,Hush Hush,46,
4,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc.",2.0,NaN
1,Jun 19 2023 9:03AM,264934,20,Reliable 1 Laboratories LLC,15.0,NaN
2,Jun 19 2023 8:49AM,264933,10,"Virtue Rx, LLC",1.0,NaN
3,Jun 19 2023 8:48AM,264932,10,Hush Hush,46.0,NaN
4,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc.",2.0,0.0
1,Jun 19 2023 9:03AM,264934,20,Reliable 1 Laboratories LLC,15.0,0.0
2,Jun 19 2023 8:49AM,264933,10,"Virtue Rx, LLC",1.0,0.0
3,Jun 19 2023 8:48AM,264932,10,Hush Hush,46.0,0.0
4,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,794796,65.0,0.0
16,264930,1.0,0.0
19,1324641,21.0,10.0
20,529855,16.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,794796,65.0,0.0
1,16,264930,1.0,0.0
2,19,1324641,21.0,10.0
3,20,529855,16.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,65.0,0.0
1,16,1.0,0.0
2,19,21.0,10.0
3,20,16.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,65.0
1,16,Released,1.0
2,19,Released,21.0
3,20,Released,16.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Executing,0.0,0.0,10.0,3.0
Released,65.0,1.0,21.0,16.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Executing,0.0,0.0,10.0,3.0
1,Released,65.0,1.0,21.0,16.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Executing,0.0,0.0,10.0,3.0
1,Released,65.0,1.0,21.0,16.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()